# AI6122 Text Management Review Summarizer
Task List:
- [ ] Define and justify what a summarizer is. E.g.
  + a list of keywords
  + a list of key phrases
  + a list of noun-adjective pairs 
  + a list of nounPhrase - adjectivePhrase pairs 
  + a list of representative sentences 
- [ ] Technical challenges to achieve ideal summarization and your solution.
- [ ] Justify approach is best option for each component in your solution.
- [ ] Justify limitations to your approach.
- [ ] Evaluate solution with possible alternative solutions (baselines).
- [ ] Randomly choose 3 products to create product review summary.

Possible algorithms to try:
- [x] (Extractive) Word frequency sentence extraction: score sentences by word freq
- (Extractive) Text rank algorithm: uses cosine similarity
- (Extractive/ Abstractive) Machine learning 



## Colab Configuration

In [0]:
from google.colab import drive
drive.mount('/content/drive')
import os 
os.chdir('/content/drive/My Drive/Colab Notebooks/txtmgmt')
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/txtmgmt


## Import Modules & Configurations

In [0]:
from collections import OrderedDict
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import pandas as pd
import multiprocessing as mp
import datetime
from helpers.duallogger import loggersetup
from helpers.filehelper import is_not_empty_file_exists, write_to_file, load_from_file
import logging

from nltk.corpus import stopwords
import nltk
import re
import heapq

In [0]:
nltk.download('punkt')
nltk.download('stopwords')
stopwords_nltk = stopwords.words('english')
stopwords_spacy = list(STOP_WORDS)
stopwords_spacy.append('\n')
stopwords = stopwords_nltk + list(set(stopwords_spacy) - set(stopwords_nltk))

print("sw nltk: ", len(stopwords_nltk))
print("sw spacy: ", len(stopwords_spacy))
print("combined: ", len(stopwords))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
sw nltk:  179
sw spacy:  327
combined:  383


In [0]:
cores = mp.cpu_count()
print("Cores:", cores)
gpu = spacy.prefer_gpu()
print("GPU:", gpu)

log_dir = './logs/'
log = loggersetup(log_dir, stdout_level=logging.DEBUG, file_level=logging.DEBUG)

# log.debug('Debug message, should only appear in the file.')
# log.info('Info message, should appear in file and stdout.')
# log.warning('Warning message, should appear in file and stdout.')
# log.error('Error message, should appear in file and stdout.')

Cores: 2
GPU: False


In [0]:
parameters = OrderedDict()
parameters['json_file'] = 'CellPhoneReview.json'
parameters['reload_prod_reviews'] = True
parameters['prod_reviews_path'] = './data/prod_reviews.data'
parameters['clean_reviews'] = True
parameters['reload_clean_reviews'] = True
parameters['cleaned_reviews_path'] = './data/prod_reviews_cleaned.data'

## Data Preprocessing

In [101]:
if not parameters['reload_prod_reviews'] or not is_not_empty_file_exists(parameters['prod_reviews_path']):
    data = pd.read_json(parameters['json_file'], lines = True)
    prod_reviews = data.groupby(['asin'])['reviewText'].apply(' '.join).reset_index()
    log.info("Writing prod_reviews to %s" % parameters['prod_reviews_path'])
    write_to_file(parameters['prod_reviews_path'], prod_reviews)
else:
    log.info("Reloading prod_reviews from %s" % parameters['prod_reviews_path'])
    prod_reviews = load_from_file(parameters['prod_reviews_path'])

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
prod_reviews.head(3) 

[INFO] Reloading prod_reviews from ./data/prod_reviews.data


asin  \
0  120401325X   
1  3998899561   
2  6073894996   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [0]:
# Create the nlp object
nlp = spacy.load("en_core_web_sm")
print(nlp.pipe_names)

['tagger', 'parser', 'ner']


In [0]:
# Define function to cleanup text by removing personal pronouns, stopwords, and punctuations
def cleanup_text(text, stopwords, punc):
    texts = []
    doc = nlp(text, disable=['parser', 'ner']) # only do tokenization and pos tagging
    tokens = [tok.lemma_.lower().strip() for tok in doc if tok.lemma_ != '-PRON-']
    tokens = [tok for tok in tokens if tok not in stopwords and tok not in punc]
    tokens = ' '.join(tokens)
    texts.append(tokens)
    return pd.Series(texts)

if parameters['clean_reviews']:
    log.debug("reviews to be stripped away with stopwords and punctation")
    if not parameters['reload_clean_reviews'] or not is_not_empty_file_exists(parameters['cleaned_reviews_path']):
      punctuations = '!"#$%&\'()*+,-/:;<=>?@[\\]^_`{|}~©'
      prod_reviews['processedText'] = prod_reviews['reviewText'].apply(lambda x: cleanup_text(x, stopwords, punctuations))
      log.info("Writing cleaned reviews to %s" % parameters['cleaned_reviews_path'])
      write_to_file(parameters['cleaned_reviews_path'], prod_reviews)
    else:
      log.info("Reloading cleaned reviews from %s" % parameters['cleaned_reviews_path'])
      prod_reviews = load_from_file(parameters['cleaned_reviews_path'])
else:
    log.info("unprocessed reviews will be used")
    prod_reviews['processedText'] = prod_reviews['reviewText']

prod_reviews['processedText'][0]

[DEBUG] reviews to be stripped away with stopwords and punctation
[INFO] Reloading cleaned reviews from ./data/prod_reviews_cleaned.data


'sticker work like review . stick great stay phone . super stylish share sister . :) awesome phone look stylish use far year believe year great quality item arrive great time perfect condition . order button great deal include free screen protector . receive . big deal nice claim come . awesome stay look great . use multiple apple product . especially nail help elevated key . use home button easy . daughter like . purchase . worth price . come describe .. come unstuck cute people ask great drive .'

## Review Summarizer

### Tokenize Words & Calculate Word Frequency 

Only calculate word frequencies of non-stopwords



In [0]:
def _word_frequency(review) -> dict: 
  global stopwords 
  word_frequencies = {}  
  for token in nlp.make_doc(review):   #tokenize
      if token.text not in stopwords:
          if token.text not in word_frequencies.keys():
              word_frequencies[token.text] = 1
          else:
              word_frequencies[token.text] += 1
  return word_frequencies

# wf = _word_frequency(prod_reviews['reviewText'][8])
# print(wf)

### Tokenize Sentences & Score Sentences

In [0]:
def _tokenize_sentences(review) -> dict:
  doc = nlp(review)
  sentence_dict = {}
  for _, sentence in enumerate(doc.sents): 
    tokenized_sent = re.sub(r'[^\w\s]','', str(sentence))
    if tokenized_sent is not None:
      sentence_dict[tokenized_sent] = len(nlp.make_doc(sentence.text))
  return sentence_dict

# sent_dict = _tokenize_sentences(prod_reviews['reviewText'][8])
# print([(key, value) for key, value in sent_dict.items()])

def _score_sentences(sentences:dict, word_frequencies:dict) -> dict:
    """Score sentences based on word frequencies"""
    sentence_scores = {}

    for sent, word_count_in_sent in sentences.items():
        for word_freq in word_frequencies:
            if word_freq in sent.lower():
                if sent in sentence_scores: # use first 10 char as key
                    sentence_scores[sent] += word_frequencies[word_freq]
                else:
                    sentence_scores[sent] = word_frequencies[word_freq]
        
        if sent in sentence_scores: # divide sentence score by word count to reduce advantages of long sentences
          sentence_scores[sent] = sentence_scores[sent] / word_count_in_sent

    return sentence_scores

# sent_scores = _score_sentences(sent_dict, wf)
# print([(key, value) for key, value in sent_scores.items()])

### Sentence-level Summarizer


In [0]:
def generate_summary(review, num_of_sentences):
    word_frequencies = _word_frequency(review)

    sentences = _tokenize_sentences(review)

    sentence_scores = _score_sentences(sentences, word_frequencies)
    
    summary_sentences = heapq.nlargest(num_of_sentences, sentence_scores, key=sentence_scores.get)
    
    return ''.join(summary_sentences)

idx = 3
num_of_sentences = 7
if parameters['clean_reviews']:
    review = prod_reviews['processedText'][idx]
    
else:
    review = prod_reviews['reviewText'][idx]

summary = generate_summary(review, num_of_sentences)
log.info("ORIGINAL REVIEW: %s", review)
log.info("SUMMARIZED REVIEW: %s", summary)

[INFO] ORIGINAL REVIEW: love case pretty . love way case feel touch rubber . happy idea design sweet idea wear paint case sealant . cool 3d effect cost paint rub . pretty . worried order picture description page change desireable green orange cover . alas come day earlier expect beautiful pink product expect rubberized cover feel little greasy like armor all'd design gorgeous little 3d look pretty phone case love . love change .thanks case white silver . pretty case- fit phone perfectly . long arrive . defintiely worth price ... order . case 15 especially cell phone store . believe mark thing great case ... fit phone perfectly use time time . recommend received item quickly . design vivid expect . cover soft rubberize durable . receive compliment . excellent buy recommend want good look phone . cover old phone look feel new . like order cover little money snazzy phone .
[INFO] SUMMARIZED REVIEW: look pretty phone case love love case pretty pretty case fit phone perfectly cover old phon